# Demo: Building Convolutional Autoencoders in Keras

Adapted from: https://blog.keras.io/building-autoencoders-in-keras.html

By: Jacob Cybulski<br>
Date: Sept 2020

Adapted From: Francois Chollet<br>
Date: 14 May 2016

## Preparation

*Standard libraries*

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

*Import tensorflow and check GPU support*

In [ ]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

**Additional note that while this notebook executes, you can watch the GPU activity using the operating system utilities, i.e.:**
- On Linux: watch -n 5 nvidia-smi
- On Windows: nvidia-smi --loop=5

**What happens when the GPU disappears? Try in this order:**
1. Restart the kernel 
2. Quit Jupyter Notebook (closing the browser is not enough)
3. Kill the "run-away" Python process
4. Restart the computer

## Data

In [ ]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.layers import UpSampling2D, BatchNormalization, LeakyReLU
from tensorflow.keras.models import Model

(x_train, _), (x_test, _) = mnist.load_data()

img_rows = 28
img_cols = 28
img_ch = 1

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

from tensorflow.keras.utils import to_categorical

# Reshape images for processing
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, img_ch)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, img_ch)
input_shape = (img_rows, img_cols, img_ch)

print(x_train.shape)
print(x_test.shape)

## Convolutional autoencoder
***Note that building autoencoder exceeds capabilities of <font color='blue'>sequential</font> models.<br>
Instead we must use <font color='blue'>functional</font> models, which can be arranged in arbitrary ways.***

A single fully-connected neural layer as encoder and as decoder:

In [ ]:
# Run settings
reg_l1 = 0
batch = 128
epochs = 300

# Model created, note these regularisations:
#   BatchNormalization prevents weight going very large or very small
#   LeakyReLU also ensures that weights do not get stuck in backpropagation

input_img = Input(shape=input_shape)

x = Conv2D(16, (3, 3), padding='same')(input_img) # activation='relu'
x = BatchNormalization()(x)
x = LeakyReLU(alpha=.001)(x)
x = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(8, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=.001)(x)
x = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(8, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=.001)(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (4, 4, 8) i.e. 128-dimensional
# why is it so large, it should be much smaller for 10 digits?

x = Conv2D(8, (3, 3), padding='same')(encoded)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=.001)(x)
x = UpSampling2D((2, 2))(x)

x = Conv2D(8, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=.001)(x)
x = UpSampling2D((2, 2))(x)

x = Conv2D(16, (3, 3))(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=.001)(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.summary()

*Create and train a model (val loss with and without reg L1 = 0.097).*<br>
*Sample callbacks have been included here, Tensorboard callbacks will slow learning.*<br>
*Unfortunately you will not be able using Tensorboard on lab machines.*<br>
*You can also experioment with different optimisers and their parameters, e.g. see:*
- Callbacks: https://www.tensorflow.org/api_docs/python/tf/keras/callbacks
- Optimisers: https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/

In [ ]:
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import SGD, RMSprop, Adadelta, Adam, Nadam

log_dir = '/tmp/autoencoder/deep-4'
callbacks = [TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True, write_images=True, 
               embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None,
               profile_batch = 100000000),
             EarlyStopping(monitor='val_loss', patience=30, verbose=0)]

opt_rmsprop = RMSprop(lr=0.005, rho=0.95, momentum=0.01, epsilon=1e-07)
opt_adadelta_1 = Adadelta(lr=0.001, rho=0.95, epsilon=1e-07)
opt_adadelta_2 = Adadelta(lr=0.05, rho=0.99, epsilon=1e-07)
opt_adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07)
opt_nadam = Nadam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07)

autoencoder.compile(optimizer=opt_nadam, loss='binary_crossentropy')
hist = autoencoder.fit(x_train, x_train,
                epochs=epochs,
                batch_size=batch,
                shuffle=True,
                validation_data=(x_test, x_test),
                callbacks=callbacks)

Plot training performance

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
def plot_hist(h, xsize=6, ysize=10):
    
    # Find what measurements were recorded
    meas = h.keys()
    
    # Prepare plotting
    fig_size = plt.rcParams["figure.figsize"]
    plt.rcParams["figure.figsize"] = [xsize, ysize]
    
    # Plot each measurement
    meas_list = []
    for m in meas:
        plt.plot(h[m])
        meas_list.append(m)

    # Add info to the plot
    ylab = ', '
    plt.ylabel(ylab.join(meas_list))
    plt.xlabel('epoch')
    plt.legend(meas_list) #, loc='upper left')
    plt.show()
    return

plot_hist(hist.history, xsize=8, ysize=5)

Show the original and predicted images

In [ ]:
# encode and decode some digits
# note that we take them from the *test* set
decoded_imgs = autoencoder.predict(x_test)

In [ ]:
# use Matplotlib
import matplotlib.pyplot as plt

n = 10  # how many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

 ## Let's see if we could now recover the original images from noisy digits

*Introduce some noise in both training and test data sets*

In [ ]:
noise_factor = 0.5
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape) 
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape) 

x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

*This is what noisy MNIST look like*

In [ ]:
n = 10
plt.figure(figsize=(20, 2))
for i in range(n):
    ax = plt.subplot(1, n, i+1)
    plt.imshow(x_test_noisy[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

*Now predict from noisy data*

In [ ]:
# encode and decode some digits
# note that we take them from the *test* set
decoded_noisy_imgs = autoencoder.predict(x_test_noisy)

# We can also try recover these digits from the training set
# decoded_noisy_imgs = autoencoder.predict(x_train_noisy[0:1000])

In [ ]:
# use Matplotlib
n = 10  # how many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test_noisy[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_noisy_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

<font color="red">**How accurate are these recovered digits?**</font>